# Hyperparameters Optimization

### Importing github repository

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("token")

!git clone https://{token}@github.com/Benedart/RecSys-2022-Challenge-Polimi.git

Cloning into 'RecSys-2022-Challenge-Polimi'...
remote: Enumerating objects: 338, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 338 (delta 52), reused 121 (delta 40), pack-reused 203
Receiving objects: 100% (338/338), 343.19 MiB | 15.68 MiB/s, done.
Resolving deltas: 100% (87/87), done.
Updating files: 100% (186/186), done.


#### Compiling Cython files

In [2]:
import os

os.chdir( "./RecSys-2022-Challenge-Polimi")
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:746:
/opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization_Cyth

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps
import csv

from Recommenders.Recommender_import_list import *

ModuleNotFoundError: No module named 'implicit'

#### Importing URM & ICM

In [4]:
URM_all = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/interactions_and_impressions.csv")
ICM_type = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_type.csv")
ICM_length = pd.read_csv("/kaggle/input/recommender-system-2022-challenge-polimi/data_ICM_length.csv")

ICM_all = pd.concat([ICM_type, ICM_length]).sort_values(by='item_id')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
URM_all

,UserID,ItemID,Impressions,Data
0,0,11,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19",1
1,0,21,NaN,0
2,0,21,NaN,0
3,0,21,"20,21,22,23,24,25,26,27,28,29",0
4,0,21,NaN,1
...,...,...,...,...
5826501,41628,20448,NaN,0
5826502,41628,20896,NaN,1
5826503,41628,21506,NaN,1
5826504,41628,22882,NaN,0


In [6]:
ICM_all

,item_id,feature_id,data
0,0,1,1
0,0,0,1
1,1,0,1
1,1,3,1
2,2,4,1
...,...,...,...
23088,27965,1,1
23089,27966,0,1
23089,27966,1,1
23090,27967,4,1


#### Getting rid of duplicates and empty indices
Let's start with the indices, the mapping is done for consistency

In [7]:
mapped_id, original_id = pd.factorize(URM_all["UserID"].unique())
user_original_Id_to_index = pd.Series(mapped_id, index=original_id)

print("Unique UserID in the URM are {}".format(len(original_id)))

mapped_id, original_id = pd.factorize(URM_all["ItemID"].unique())

print("Unique ItemID in the URM are {}".format(len(original_id)))

all_item_indices = pd.concat([URM_all["ItemID"], ICM_all["item_id"]], ignore_index=True)
mapped_id, original_id = pd.factorize(all_item_indices.unique())

print("Unique ItemID in the URM and ICM are {}".format(len(original_id)))

item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(ICM_all["feature_id"].unique())
feature_original_ID_to_index = pd.Series(mapped_id, index=original_id)

print("Unique FeatureID in the URM are {}".format(len(feature_original_ID_to_index)))

URM_all["UserID"] = URM_all["UserID"].map(user_original_Id_to_index)
URM_all["ItemID"] = URM_all["ItemID"].map(item_original_ID_to_index)
ICM_all["item_id"] = ICM_all["item_id"].map(item_original_ID_to_index)
ICM_all["feature_id"] = ICM_all["feature_id"].map(feature_original_ID_to_index)

Unique UserID in the URM are 41629
Unique ItemID in the URM are 24507
Unique ItemID in the URM and ICM are 27968
Unique FeatureID in the URM are 6


And then we remove duplicate values

In [8]:
URM_all = URM_all.drop(["Impressions", "Data"], axis=1).drop_duplicates()
URM_all["Rating"] = 1
ICM_all = ICM_all.drop_duplicates()

In [9]:
URM_all

,UserID,ItemID,Rating
0,0,0,1
1,0,1,1
13,0,2,1
28,0,3,1
29,0,4,1
...,...,...,...
5826501,41628,3700,1
5826502,41628,12694,1
5826503,41628,15132,1
5826504,41628,18128,1


In [10]:
ICM_all

,item_id,feature_id,data
0,3668,0,1
0,3668,1,1
1,1945,1,1
1,1945,2,1
2,18373,3,1
...,...,...,...
23088,27965,0,1
23089,27966,1,1
23089,27966,0,1
23090,27967,3,1


#### Creating the matrices

In [11]:
n_users = len(user_original_Id_to_index)
n_items = len(item_original_ID_to_index)
n_features = len(feature_original_ID_to_index)

from sklearn.model_selection import train_test_split

seed = 42

(user_ids_training_validation, user_ids_test,
 item_ids_training_validation, item_ids_test,
 ratings_training_validation, ratings_test) = train_test_split(URM_all.UserID,
                                                    URM_all.ItemID,
                                                    URM_all.Rating,
                                                    test_size=0.20,
                                                    shuffle=True,
                                                    random_state=seed)

(user_ids_training, user_ids_validation,
 item_ids_training, item_ids_validation,
 ratings_training, ratings_validation) = train_test_split(user_ids_training_validation,
                                                          item_ids_training_validation,
                                                          ratings_training_validation,
                                                          test_size=0.20,
                                                          shuffle=True,
                                                          random_state=seed)

URM_all = sps.csr_matrix((URM_all.Rating, (URM_all.UserID, URM_all.ItemID)),
                           shape = (n_users, n_items))

URM_train = sps.csr_matrix((ratings_training, (user_ids_training, item_ids_training)),
                           shape = (n_users, n_items))

URM_validation = sps.csr_matrix((ratings_validation, (user_ids_validation, item_ids_validation)),
                           shape = (n_users, n_items))

URM_train_validation = sps.csr_matrix((ratings_training_validation, (user_ids_training_validation, item_ids_training_validation)),
                           shape = (n_users, n_items))

URM_test = sps.csr_matrix((ratings_test, (user_ids_test, item_ids_test)),
                           shape = (n_users, n_items))


ICM_all = sps.csr_matrix((np.ones(len(ICM_all["item_id"].values)), 
                          (ICM_all["item_id"].values, ICM_all["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all.data = np.ones_like(ICM_all.data)

ICM_all = sps.csr_matrix(ICM_all)
features_per_item = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csc_matrix(ICM_all)
items_per_feature = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csr_matrix(ICM_all)

#### Setting up evaluators

In [12]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 753 ( 1.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 323 ( 0.8%) Users that have less than 1 test interactions


### Models

In [13]:
SLIM_Elastic = SLIMElasticNetRecommender(URM_train_validation)
SLIM_Elastic.fit(
    topK=5325, 
    l1_ratio= 1e-05, 
    alpha= 0.001
)

SLIMElasticNetRecommender: URM Detected 3461 (12.4%) items with no interactions.
SLIMElasticNetRecommender: Processed 1324 ( 4.7%) in 5.00 min. Items per second: 4.41
SLIMElasticNetRecommender: Processed 3310 (11.8%) in 10.00 min. Items per second: 5.51
SLIMElasticNetRecommender: Processed 5522 (19.7%) in 15.00 min. Items per second: 6.13
SLIMElasticNetRecommender: Processed 8015 (28.7%) in 20.00 min. Items per second: 6.68
SLIMElasticNetRecommender: Processed 10559 (37.8%) in 25.00 min. Items per second: 7.04
SLIMElasticNetRecommender: Processed 13235 (47.3%) in 30.00 min. Items per second: 7.35
SLIMElasticNetRecommender: Processed 16018 (57.3%) in 35.01 min. Items per second: 7.63
SLIMElasticNetRecommender: Processed 18767 (67.1%) in 40.01 min. Items per second: 7.82
SLIMElasticNetRecommender: Processed 21528 (77.0%) in 45.01 min. Items per second: 7.97
SLIMElasticNetRecommender: Processed 23791 (85.1%) in 50.01 min. Items per second: 7.93
SLIMElasticNetRecommender: Processed 24776 (

In [15]:
ItemKNN_CFCBF = ItemKNN_CFCBF_Hybrid_Recommender(URM_train_validation, ICM_all)
ItemKNN_CFCBF.fit(
    topK= 40, 
    shrink= 34, 
    similarity= 'tversky', 
    normalize= True, 
    tversky_alpha= 0.1911656636857864, 
    tversky_beta= 1.4823556453405908, 
    ICM_weight= 0.026917911342859663
)

ItemKNN_CFCBF_HybridRecommender: URM Detected 3461 (12.4%) items with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 4877 (17.4%) items with no features.
Similarity column 27968 (100.0%), 1413.50 column/sec. Elapsed time 19.79 sec


In [16]:
RP3Beta = RP3betaRecommender(URM_train_validation)
RP3Beta.fit(
    topK= 135,
    alpha= 0.8473968835576251, 
    beta= 0.25814662527577076,
    normalize_similarity= True
)

RP3betaRecommender: URM Detected 3461 (12.4%) items with no interactions.
RP3betaRecommender: Similarity column 27968 (100.0%), 1773.82 column/sec. Elapsed time 15.77 sec


#### Combining the models into the hybrid

In [18]:
recommenders = [SLIM_Elastic, ItemKNN_CFCBF, RP3Beta] 
alphas = [2.0, 0.1523307518368696, 2.0]

recommender = GeneralizedLinearHybridRecommender(
    URM_train = URM_train_validation, 
    recommenders=recommenders
)

recommender.fit(alphas)

SLIMElasticNetItemKNN_CFCBF_HybridRP3betaHybridRecommender: URM Detected 3461 (12.4%) items with no interactions.


In [22]:
evaluator_test.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 41306 (100.0%) in 1.32 min. Users per second: 520


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                     
 10      0.069612                 0.095166  0.08263  0.035856    0.048042   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.217853  0.098936  0.075565  0.415412      0.276454  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.992241          0.412189    0.992241       0.015762   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            8.638782                   0.992176             0.035417   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                      

## Submission
### Preparing the submission
<br>
Computing the original indexes

In [54]:
target = pd.read_csv(filepath_or_buffer="/kaggle/input/recommender-system-2022-challenge-polimi/data_target_users_test.csv",
                                          sep=",",
                                          header=0,
                                          engine='python',
                                          names=['id'])

In [41]:
def write_recommendations(recommender, file_name):
    recommendations = 'user_id,item_list'
    f = open(file_name + ".csv", "w")

    for id in target['id']:
        recommendations_per_user = recommender.recommend(user_id_array=id, remove_seen_flag=True, cutoff=10)

        recommendation_string = str(user_original_Id_to_index[user_original_Id_to_index==id].index.item()
                                    ) + ','

        for rec in recommendations_per_user:
            recommendation_string = recommendation_string + str(item_original_ID_to_index[item_original_ID_to_index==rec].index.item()) + ' '

        recommendation_string = recommendation_string[:-1]
        recommendations = recommendations + '\n' + recommendation_string

    f.write(recommendations)
    f.close()

In [60]:
write_recommendations(recommender = recommender, file_name = "submission")

In [61]:
my_submission = pd.read_csv("submission.csv")
my_submission

,user_id,item_list
0,0,22 617 18484 752 23 20 29 2488 25 673
1,1,20420 6874 61 56 665 393 352 450 62 1068
2,2,7864 121 8819 453 6712 21319 18939 1311 1308 2...
3,3,21 20 58 269 352 613 917 53 130 169
4,4,23 21 80 68 56 124 348 2106 541 64
...,...,...
41111,41624,7864 22403 10221 18939 617 11044 22441 8819 11...
41112,41625,53 21 23 25 393 56 270 20 596 450
41113,41626,21 124 12118 23 393 53 55 20 22619 3546
41114,41627,1514 1214 19705 17880 1561 1212 1859 71 4577 5069
